In [1]:
import pandas as pd 
from model_loader import model_loader
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
from sklearn.metrics import pairwise_distances_argmin_min

In [2]:
df=pd.read_csv("/home/mario/Documents/Facultate/Licenta/Code/Back End/data/categorised_data/papers_categorised.csv")
df

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID,keyword,Category,SVD
0,Maciel D.A.; Lousada F.; Fassoni-Andrade A.; P...,"Maciel, Daniel A (57190374434); Lousada, Felli...",57190374434; 59412677500; 56815026900; 5915894...,Sentinel-1 data reveals unprecedented reductio...,2024,Environmental Research Letters,19,12,124034,NaN,...,English,Environ.Res.Lett.,Article,Final,NaN,Scopus,2-s2.0-85209260114,ClimateChange/ClimateChange1,103.0,"[0.009274202398955822, 0.1082928404211998, 0.0..."
1,Gong J.; Yang G.; Zhang S.; Zhang W.; Dong X.;...,"Gong, Jirui (7402707592); Yang, Guisen (572115...",7402707592; 57211567507; 57343096700; 57202425...,Human activities weaken the positive effects o...,2024,Science of the Total Environment,957,NaN,177564,NaN,...,English,Sci. Total Environ.,Article,Final,NaN,Scopus,2-s2.0-85210303746,ClimateChange/ClimateChange1,134.0,"[0.019757576286792755, 0.016898566856980324, -..."
2,Cordero R.R.; Feron S.; Damiani A.; MacDonell ...,"Cordero, Raúl R. (8596522500); Feron, Sarah (5...",8596522500; 57189441518; 59123676800; 24759068...,Rapid decline in extratropical Andean snow cov...,2024,Scientific Reports,14,1,26365,NaN,...,English,Sci. Rep.,Article,Final,All Open Access; Gold Open Access; Green Open ...,Scopus,2-s2.0-85209475509,ClimateChange/ClimateChange1,218.0,"[0.029680419713258743, 0.014101012609899044, -..."
3,Paul S.; Das C.S.; Chaudhuri S.,"Paul, Swapan (57327861400); Das, Chandan Surab...",57327861400; 47661072600; 57224730679,Seasonal vertical accretion in different hydro...,2024,Geomorphology,467,NaN,109493,NaN,...,English,Geomorphology,Article,Final,NaN,Scopus,2-s2.0-85207901321,ClimateChange/ClimateChange1,195.0,"[0.044876378029584885, 0.0911358967423439, 0.0..."
4,Liu Z.; Yan J.; Dong H.; Zhang Q.; Zhang S.,"Liu, Zhaopu (58998148700); Yan, Jianzhong (833...",58998148700; 8337874400; 57208753866; 57542260...,The impact of policy measures on livelihood di...,2024,Mitigation and Adaptation Strategies for Globa...,29,8,85,NaN,...,English,Mitigation Adapt. Strateg. Global Change,Article,Final,NaN,Scopus,2-s2.0-85209726080,ClimateChange/ClimateChange1,134.0,"[-0.03704876825213432, -0.01357633899897337, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55584,Wang T.; Huang D.; Shen J.; Chen Y.; Sun G.; W...,"Wang, Teng (57281934600); Huang, Dan (55183724...",57281934600; 55183724800; 55726264500; 2417075...,Batch fecundity and spawning frequency of inva...,2014,Environmental Biology of Fishes,97,10,NaN,1161,...,English,Environ. Biol. Fishes,Article,Final,NaN,Scopus,2-s2.0-84887639403,NaN,292.0,"[-0.01097214687615633, 0.0202456284314394, -0...."
55585,Johnson N.S.; Swink W.D.; Brenden T.O.; Slade ...,"Johnson, Nicholas S. (8308237400); Swink, Will...",8308237400; 6603235356; 57203407751; 710241374...,Survival and metamorphosis of low-density popu...,2014,Journal of Great Lakes Research,40,1,NaN,155,...,English,J. Great Lakes Res.,Article,Final,NaN,Scopus,2-s2.0-84895862076,NaN,217.0,"[-0.007852651178836823, 0.03381326049566269, -..."
55586,Harvey G.L.; Henshaw A.J.; Moorhouse T.P.; Cli...,"Harvey, Gemma L (25821590000); Henshaw, Alexan...",25821590000; 46062022700; 8704131700; 67016709...,Invasive crayfish as drivers of fine sediment ...,2014,Earth Surface Processes and Landforms,39,2,NaN,259,...,English,Earth Surf. Processes Landf.,Article,Final,All Open Access; Green Open Access,Scopus,2-s2.0-84892681327,NaN,188.0,"[-0.03408987820148468, 0.009875274263322353, 0..."
55587,Karosiene J.; Kasperovičiene J.; Koreiviene J....,"Karosiene, Jurate (49361242300); Kasperovičien...",49361242300; 6504170818; 6506681563; 55999456300,Assessment of the vulnerability of Lithuanian ...,2014,Limnologica,45,NaN,NaN,7,...,English,Limnologica,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-8489216124

In [1]:
def keyword_getter(files):
    df = pd.DataFrame()
    for file in files:
        file_df = pd.read_csv(file)
        df=pd.concat([df,file_df])

    author_keywords = df["Author Keywords"]
    author_keywords = list(author_keywords)

    formatted_list = []

    for i in author_keywords:
        if isinstance(i, str):
            for j in i.split(";"):
                formatted_list.append(str(
                    j.strip()
                    .replace("<sub>", "")
                    .replace("</sub>", "")
                    .replace("<sup>", "")
                    .replace("</sup>", "")
                    .lower()
                ))

    formatted_list = list(set(formatted_list))
    formatted_list.remove("")
    return formatted_list

In [3]:
documents_list = [
    "/home/mario/Documents/Facultate/Licenta/Code/Back End/data/climateChange.csv",
    "/home/mario/Documents/Facultate/Licenta/Code/Back End/data/EcosystemsServices.csv",
    "/home/mario/Documents/Facultate/Licenta/Code/Back End/data/Eutrophication.csv",
    "/home/mario/Documents/Facultate/Licenta/Code/Back End/data/InvasiveSpecies.csv",
]
keywords = keyword_getter(documents_list)

/tmp/ipykernel_46910/4187536017.py:4: DtypeWarning: Columns (8,9,10,21) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file)


In [14]:
my_dict = {i: [] for i in range(100)}
tlfif, kmeans, svd = model_loader()
keyword=tlfif.transform(["river"])
keyword=svd.transform(keyword)
closest_cluster_index, _ = pairwise_distances_argmin_min(keyword, kmeans.cluster_centers_)
cluster_id = int(closest_cluster_index[0])
print(cluster_id)

4


In [33]:
tfidf_model= TfidfVectorizer()
categorised_data = pd.DataFrame({"keyword": keywords})
keywords_encoded = tfidf_model.fit_transform(categorised_data["keyword"])
svd = TruncatedSVD(n_components=100, random_state=42)
X_reduced = svd.fit_transform(keywords_encoded) 
n_clusters = 100
kmeans_model = KMeans(n_clusters=n_clusters, init='k-means++')
kmeans_model.fit(X_reduced)

KMeans(n_clusters=100)

In [ ]:
def categoriser(files):
    df = pd.DataFrame()
    for file in files:
        file_df = pd.read_csv(file)
        df = pd.concat([df, file_df])

    author_keywords = df[["Author Keywords", "DOI"]]
    author_keywords["Author Keywords"] = author_keywords["Author Keywords"].astype(str)
    author_keywords["Author Keywords"] = author_keywords["Author Keywords"].apply(
        lambda x: ";".join(
            [
                keyword.strip()
                .replace("<sub>", "")
                .replace("</sub>", "")
                .replace("<sup>", "")
                .replace("</sup>", "")
                .lower()
                for keyword in x.split(";")
                if pd.notna(x)
            ]
        )
    )

    author_keywords.loc["Category"] = np.nan                                    
    author_keywords = author_keywords[pd.notna(author_keywords["Author Keywords"])]

    for i, row in tqdm(
        author_keywords.iterrows(),
        total=author_keywords.shape[0],
        desc="Categorising articles",
    ):
        keywords_encoded = tfidf_model.transform([row["Author Keywords"]])
        svd_reduced= svd.transform(keywords_encoded)
        word_categories = list(kmeans_model.predict(svd_reduced))
        category = max(word_categories, key=lambda x: word_categories.count(x))
        author_keywords.loc[i, "Category"] = category

    author_keywords = author_keywords[["DOI", "Category"]]
    author_keywords = author_keywords[pd.notna(author_keywords["DOI"])]

    df["Category"] = np.nan
    for _, row in tqdm(
        author_keywords.iterrows(),
        total=author_keywords.shape[0],
        desc="Joining tables",
    ):
        index = df.index[df["DOI"] == row["DOI"]].to_list()[0]
        df.loc[index, "Category"] = row["Category"]
    
    print(author_keywords)
    print(df)
    return author_keywords, df

In [37]:
author_keywords, categorised_articles = categoriser(documents_list)
author_keywords, categorised_articles

/tmp/ipykernel_18435/4144477650.py:4: DtypeWarning: Columns (8,9,10,21) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file)
/tmp/ipykernel_18435/4144477650.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_keywords["Author Keywords"] = author_keywords["Author Keywords"].astype(str)
/tmp/ipykernel_18435/4144477650.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_keywords["Author Keywords"] = author_keywords["Author Keywords"].apply(
/tmp/

                                  DOI  Category
0            10.1088/1748-9326/ad8a71      64.0
1     10.1016/j.scitotenv.2024.177564       1.0
2          10.1038/s41598-024-78014-0       1.0
3      10.1016/j.geomorph.2024.109493      50.0
4          10.1007/s11027-024-10182-w      50.0
...                               ...       ...
4831        10.1007/s10641-013-0205-8       1.0
4832       10.1016/j.jglr.2013.12.005       1.0
4833                 10.1002/esp.3486      50.0
4834      10.1016/j.limno.2013.10.005      50.0
4835       10.1016/j.jglr.2014.01.004       1.0

[54135 rows x 2 columns]
                                                Authors  \
0     Maciel D.A.; Lousada F.; Fassoni-Andrade A.; P...   
1     Gong J.; Yang G.; Zhang S.; Zhang W.; Dong X.;...   
2     Cordero R.R.; Feron S.; Damiani A.; MacDonell ...   
3                       Paul S.; Das C.S.; Chaudhuri S.   
4           Liu Z.; Yan J.; Dong H.; Zhang Q.; Zhang S.   
...                                         

(                                  DOI  Category
 0            10.1088/1748-9326/ad8a71      64.0
 1     10.1016/j.scitotenv.2024.177564       1.0
 2          10.1038/s41598-024-78014-0       1.0
 3      10.1016/j.geomorph.2024.109493      50.0
 4          10.1007/s11027-024-10182-w      50.0
 ...                               ...       ...
 4831        10.1007/s10641-013-0205-8       1.0
 4832       10.1016/j.jglr.2013.12.005       1.0
 4833                 10.1002/esp.3486      50.0
 4834      10.1016/j.limno.2013.10.005      50.0
 4835       10.1016/j.jglr.2014.01.004       1.0
 
 [54135 rows x 2 columns],
                                                 Authors  \
 0     Maciel D.A.; Lousada F.; Fassoni-Andrade A.; P...   
 1     Gong J.; Yang G.; Zhang S.; Zhang W.; Dong X.;...   
 2     Cordero R.R.; Feron S.; Damiani A.; MacDonell ...   
 3                       Paul S.; Das C.S.; Chaudhuri S.   
 4           Liu Z.; Yan J.; Dong H.; Zhang Q.; Zhang S.   
 ...                   

In [38]:
print(categorised_articles)

                                                Authors  \
0     Maciel D.A.; Lousada F.; Fassoni-Andrade A.; P...   
1     Gong J.; Yang G.; Zhang S.; Zhang W.; Dong X.;...   
2     Cordero R.R.; Feron S.; Damiani A.; MacDonell ...   
3                       Paul S.; Das C.S.; Chaudhuri S.   
4           Liu Z.; Yan J.; Dong H.; Zhang Q.; Zhang S.   
...                                                 ...   
4831  Wang T.; Huang D.; Shen J.; Chen Y.; Sun G.; W...   
4832  Johnson N.S.; Swink W.D.; Brenden T.O.; Slade ...   
4833  Harvey G.L.; Henshaw A.J.; Moorhouse T.P.; Cli...   
4834  Karosiene J.; Kasperovičiene J.; Koreiviene J....   
4835  Shea T.B.; Sheath R.G.; Chhun A.; Vis M.L.; Ch...   

                                      Author full names  \
0     Maciel, Daniel A (57190374434); Lousada, Felli...   
1     Gong, Jirui (7402707592); Yang, Guisen (572115...   
2     Cordero, Raúl R. (8596522500); Feron, Sarah (5...   
3     Paul, Swapan (57327861400); Das, Chandan Surab...